<a href="https://colab.research.google.com/github/npd2013/Mypublicmodels/blob/main/Experiment_6_1f_using_TFIDF_with_spell_check_dictionary__saluteremove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SETUP1:-  
-Single document text summarization 

-frequency based method for scoring sentences 

-preprocessing applied are knowledge based normalization,  stopword removal and stemming.
-NO CUE WORDS 

-automatic evaluation using rouge and compression ratio analysis.

note:results stored in another file 


Algorithm steps :- 

1) Preprocessing of text- 

    1.1)Cleaning 

    1.1.1)Removal of unnecessary symbols/charachters like html tags ,extra spaces,extra newlines etc.
         
 
 1.2) Line segmentation
 
    -using nltk-punkt 
    -Divide text into lines.
 1.3) word segmentation
    -Divide the lines into words.
    -words can be separated by whitespace or characters like quotes or comma.  use the punctuation list to exclude from the word.Only extract valid words 

  1.4) Stopword removal 
       -basic set of 15 words removed from lines. 
   
 1.5) Stemming
  use indowordnet as dictionary of standard words and prefix,suffix AND replacement longest applicable rules for finding stem of words.Also if by trimming size of word becomes less than 3, it is considered to be overstemmed ,so ignored.  
                   
               
       

2)Make vector space representation -based on term frequency

    2.1) Compute tfidf using sklearn module of python.

    2.2) For each sentence - 

     2.2.1) Compute sentence score= total tfidf score of all words in sentence divide by number of words in sentence.
     2.2.2) If it is first or last sentence , increase its weightage ten times.
  
  2.3) For each sentence -

     3.2.1) Compute the average normalized score  : 
          Current sentence score divide by total sentence score of text.
      

 4) //Find top "n" scored sentences.

 4.1) Set n= number of lines in gold summary. 0r "x" percent of original text.    
 4.2) Summary_length=n
 4.3) Sorted_sentences=sort(sentences) based on their scores.
    
 5) //Generate the summary -

    for sentences = 1 to gold_summary_length 
      - if sentence in gold_summary  
           put sentence in system_summary. 
  
6)// Evaluate the summary-

    6.1) use the Rouge score 
      - Get the precision ,recall and F-measure for rouge-1 ,rouge2 and rouge-LCS.   

    6.2) Find comression ratio of text
       6.2.1) sentence compression ratio
        = total sentences in system_summary/total sentences in source text 
        6.2.2) Word compression ratio
        =total words in system_summary/total words in source 
  


In [1]:
#install needed packages 
#!pip install  gensim  #has word2vec model built, using english 

!pip install pyiwn
!pip install -U scikit-learn
!pip install rouge
!pip install indic-nlp-library


Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.24.1)


In [2]:
!cp "/content/drive/My Drive/Colab Notebooks/guj_final_preporcess-v6.py" "guj_final_preporcess.py" #The preprocessing modules designed
!cp "/content/drive/My Drive/Colab Notebooks/Evaluation.py" "Evaluation.py"   # the evaluation module designed
!cp "/content/drive/My Drive/Colab Notebooks/guj_final_postprocessing-v1.py" "post.py"   # Get the post processing module 


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
import io
import csv
from collections import Counter
from Evaluation import *
from post import *
import pandas as pd 
from guj_final_preporcess import * #to get the required preporcessing modules -from self made pipeline
import pyiwn

In [9]:
from google.colab import files #upload the files from local machine on the colab environment 
uploaded = files.upload()

Saving story1.txt to story1 (1).txt
Saving story1-gold.txt to story1-gold (1).txt
Saving story1-topic.txt to story1-topic.txt
Saving story2.txt to story2.txt
Saving story2-gold.txt to story2-gold.txt
Saving story2-topic.txt to story2-topic.txt
Saving story3.txt to story3.txt
Saving story3-gold.txt to story3-gold.txt
Saving story3-topic.txt to story3-topic.txt
Saving story4.txt to story4.txt
Saving story4-gold.txt to story4-gold.txt
Saving story4-topic.txt to story4-topic.txt
Saving story5.txt to story5.txt
Saving story5-gold.txt to story5-gold.txt
Saving story5-topic.txt to story5-topic.txt
Saving story6.txt to story6.txt
Saving story6-gold.txt to story6-gold.txt
Saving story6-topic.txt to story6-topic.txt
Saving story7.txt to story7.txt
Saving story7-gold.txt to story7-gold.txt
Saving story7-topic.txt to story7-topic.txt
Saving story8.txt to story8.txt
Saving story8-gold.txt to story8-gold.txt
Saving story8-topic.txt to story8-topic.txt
Saving story9.txt to story9.txt
Saving story9-go

In [150]:

#take all the needed inputs from user 
basename=input('Enter the basic name of file(without txt),other names wud be autogenerated  ')
filename=basename+'.txt'
gold_filename=basename+'-gold.txt'
#filename=input('Enter text file name to be summarized ') #Ex: file1.txt
#analysefile_name=filename.replace('.txt','analysis-TF+ALL.txt')# to store the data for later analysis . Example- file1-results.txt 
#features_filename=filename.replace('.txt','TF+ALL-features.csv') # file-1-features.csv
#gold_filename=input("Enter name of file containing gold summary  ") #example file1-gold.txt ,assumes the name is having pattern ending with gold.txt 
summary_filename=filename.replace('gold.txt','-TF+ALL_systemsummary-topic based.txt') # example file1-gold-system-summary.txt


Enter the basic name of file(without txt),other names wud be autogenerated  story35


In [151]:
#read the input original file . 


guj_clean_html(filename,filename)# text cleaning remove the html tags and overwrite the original file 
guj_clean_html(gold_filename,gold_filename)# clean the html tags from gold file 

#Display the given data in input files 
with io.open(filename,'r',encoding='utf-8') as f: 
		  lines=f.read()#get all the lines from file
analyzedfile_obj=open(analysefile_name,mode='w',encoding='UTF-8')
analyzedfile_obj.write("\n\n The original file is :------\n")
analyzedfile_obj.write(lines)

with io.open(gold_filename,'r',encoding='utf-8') as f: 
		  lines=f.read()#get all the lines from file

analyzedfile_obj.write("\n\n The gold summary provided by expert  is -->\n")    
analyzedfile_obj.write(lines)




# now find the statistics of originally given informations 
originalcorpora,totaloriginalsentences=guj_corpus_generate(filename) #make corpus of original data 
goldcorpora,totalgoldcorpora=guj_corpus_generate(gold_filename) #make corpus of gold 

analyzedfile_obj.write("\n\n STATISTICS------>")
analyzedfile_obj.writelines("\n Original file sentences(RAW) - {0}\n".format(totaloriginalsentences))
analyzedfile_obj.writelines("\n gold file sentences(RAW)- {0}\n".format(totalgoldcorpora))

###################################################################
########################  Step-1 ##################################
#####################  cleaning phase  ########################

#doing cleaning of file 
#remove newlines and other unnecessary data and clean it for feature extraction 
#to replace the data as per our format, replacing abbreviations dot with '*'
normfile="temp1.txt"
norm_gold_file="temp2.txt"

guj_clean_text(filename,normfile) 
guj_clean_text(gold_filename,norm_gold_file)


#Now display the stats and make corpora after cleaning file and removing new lines . 
normcorpora,totalnormsentences=guj_corpus_generate(normfile)  
goldcorpora,totalnormgoldsentences=guj_corpus_generate(norm_gold_file) 


analyzedfile_obj.write(" Segmented Lines in source -{0} \n ".format(totalnormsentences))
analyzedfile_obj.write(" Segmented Lines in gold -{0}\n ".format(totalnormgoldsentences))

###################################################################
########################  Step-2 ##################################
#####################  Normalization and tokenization  for feature extraction  ########################

cleanfile="file.txt"
mydictionary=guj_makedictionary()
guj_stopwordremoval(normfile,cleanfile) #remove stopwords
guj_stemmer_new(cleanfile,cleanfile,mydictionary)




#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#####


#Now working on the modified file to get various features 

corpora,totalsent=guj_corpus_generate(cleanfile) #divide lines into sentences-
analyzedfile_obj.write(" Lines in source after all processing -{0} \n ".format(totalsent))

sent_pos_list=[] #to store the sentence position
sent_words_count=[]  # to store the total words in each sentence 
sent_number=0
for s in corpora:
  sent_pos_list.append(sent_number)
  sent_number+=1
 # print("\nsentence is ",s)
  tokens=guj_mytokenizer(s)
 # print("\ntokens are ",tokens)
  

# to generate vector based on frequency 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer = guj_mytokenizer,min_df=0.15) #frequency of words with atleast 15% frequency

cwm = vectorizer.fit_transform(corpora) #vectorize the corpus of input file

tokens = vectorizer.get_feature_names() #get the names of words used in vector
print("tokens / words extracted are ",tokens)
counts=cwm.toarray().sum(axis=1)
counts=cwm.sum(axis=0).A1
freqwithwords=dict(zip(tokens,counts))
analyzedfile_obj.write("\n\n****The TFIDF  based vector is formed ****")
analyzedfile_obj.write('\n The dimension of vector matrix generated is -\n {0} '.format(cwm.shape))
analyzedfile_obj.write('\n\n Vocabulory of words with frequencies are  -  \n {0} '.format(freqwithwords,2))
#to store the dataset in csvfile
print("Original freq values of words considered were ",freqwithwords)
 
 
list_of_tuples = list(zip(sent_pos_list,corpora, cwm.toarray()))  
#generate a dataframe to store the data 
import pandas as pd 
df=pd.DataFrame(list_of_tuples, columns=['sent_pos','sentence','frequency_vector'])
df.to_csv(features_filename) 



##########################################################
################# ######Step3#############################
################ Compute sentence scores #################
##########################################################
## Individual Sentence scoring 


score_list=[] #to store sentencewise scores

total_allsent_scores=0
for s in corpora:
  words=0
  sentenceval=0 #to store the score of sentence 
  
  for word in guj_mytokenizer(s): #use own tokenizer
    for key,value in freqwithwords.items():
      if key ==word:
        sentenceval=sentenceval+value
        words=words+1 #important word in current sentence increased
  sentenceval = float(f"{sentenceval :.2f}") #rounding off to two numbers
  if words>0:
    sentenceval=sentenceval/words #find the average score of sentence ,to normalize for long sentences
  score_list.append(sentenceval)    #store the direct value of sentence based on words found in it   
  total_allsent_scores=total_allsent_scores+sentenceval #update the total score of all corpus
      
avg_sent_score=total_allsent_scores/len(corpora) #find average score of sentence in corpus    
newscorelist=[]

pos=0 #position index in dataframes start from 0
for score in score_list:
  #score=score/avg_sent_score #normalize by average score 
  score=score/total_allsent_scores #normalize by total score ,gives better accuracy
  sent_position=df.iat[pos,0] #fetch the sentence position ,to update special weight to first and last line 
  if sent_position==0 or sent_position==totalsent-1:#as indexing from zero ,decrease by 1 
    score=1 #increase its importance to 100%
  newscorelist.append(score)
  pos=pos+1 #to get the next sentence from frame


       

analyzedfile_obj.write("\n\n The individual scores of all sentences are :- \n {0}".format(score_list,2))
analyzedfile_obj.write("\n\n The normalized scores  (based on average score ) are {0} \n ".format(newscorelist,2))
#df['sentence_wordcount']=word_count_list
df['sent_score']=score_list         #add the new collumn of individual sentence normalized score based onits length
df['normalized_score']=newscorelist # store score based on average score of all sentences in text
df.to_csv(features_filename) 



2021-02-22:13:16:16,506 INFO     [iwn.py:43] Loading gujarati language synsets...


original word was  ભુ
updated word after  normalizeing ભૂ
original word was  કૂ
updated word after  normalizeing કુ
original word was  કૂ
updated word after  normalizeing કુ
original word was  ભુ
updated word after  normalizeing ભૂ
tokens / words extracted are  ['એમણ', 'ચોરી', 'જ', 'જાતે', 'તરવાડ', 'દલ', 'ભૂએ', 'લઇ', 'વશરામ', 'વાડી', 'શાકભાજી']
Original freq values of words considered were  {'એમણ': 7.478856748649475, 'ચોરી': 3.4445697543079654, 'જ': 4.645022527795701, 'જાતે': 3.225862516560162, 'તરવાડ': 4.694642245260513, 'દલ': 5.987692384057975, 'ભૂએ': 3.9148916311957622, 'લઇ': 3.2809930189218455, 'વશરામ': 6.538579205476664, 'વાડી': 6.789246734068492, 'શાકભાજી': 4.01852838885774}


In [152]:

rougescorefile="Rougeresults-all-TF.csv"
rougescorefilehandle=open(rougescorefile,mode='a',encoding="utf-8") #to store only the rouge scores for later analysis 

resultvar = csv.writer(rougescorefilehandle, lineterminator='\n')		#to store new row for each f  

##########################################################
################# ######Step4#############################
################ Find top sentences  #################
##########################################################
#rank and sort the sentences  based on individual sentences score 

optimalratio=(totalnormgoldsentences/totalnormsentences)*100 #ideal ratio as per gold text given 
optimalratio = float(f"{optimalratio :.2f}")
data=df.sort_values(["sent_score"],ascending=False,kind='quicksort')
resultlist=[]#to store the rouge score data
for percent in [optimalratio]:
  resultlist.append("Extraction ratio"+str(percent)+",")
  analyzedfile_obj.write("\n\nThe method of summary is *** {0}% *** of original text".format(percent))
  n= round(totalnormsentences*int(percent)/100)
  top_sent=data.iloc[0:n,1:2]  
 # print(top_sent.head())
  toplist=list(top_sent.sentence)
 
  analyzedfile_obj.write("\n\n List of top sentences are as follows-{0} ".format(toplist))
  ##########################################################
################# ######Step6#############################
################ Generate summary  #################
##########################################################
  summary=[]
  not_in_summary=[]
  sentence_number=0
  for sent in corpora:  #take one by one the sentences considered in feature vector 
    if sent in toplist: #if sentence is in topranked sentences then store its number so its original line can be fetched from original corpus
        summary.append(sentence_number)
    else:
        not_in_summary.append(sentence_number)
    sentence_number=sentence_number+1

  analyzedfile_obj.write("\n\n ***** The summary geneated has the following numbered sentences ---->>\n {0}".format(summary))
  analyzedfile_obj.write("\n\n ***** The sentences not included in summary are these numbers---->>{0}".format(not_in_summary))
  
  counter=0 #now fetching the original lines to be copied in final summary. 
  summary_lines=[]
  for sent in normcorpora:
    if counter in summary:
      summary_lines.append(sent)   
    counter=counter+1

 
  #store the summary in file
  summary_filename='system-temp-{0}.txt'.format(str(percent))
    #to give name tosummary file for storing percent based results
  #format is  original file name -system-<percent> Ex:- news1-system-50
  
  with io.open(summary_filename,'w',encoding='utf-8') as w: 
    for i in summary_lines:
      w.write(i)
  
  ###########step5#############
  ########### Post Processing ######
  resultfilename="system_summary_TF+stopwordr+stem_"+filename
  
  post_process(summary_filename,resultfilename)#do the post processing and write in result file
################# ######Step6#############################
################ Evaluate summary  #################
  guj_remove_newlines(norm_gold_file,norm_gold_file)
  guj_remove_newlines(summary_filename,summary_filename)
  rougescores=evalate_files(norm_gold_file,summary_filename)#get the evaluation of generated summary after post processing based on rouge score
  for key in rougescores.keys():
     resultlist.append("%s,%s"%(key,rougescores[key]))
  resultvar.writerow(resultlist)   
  analyzedfile_obj.write("\n **********************Rouge scores based analysis for {0} % extraction is as follows*********************** ".format(percent))
  analyzedfile_obj.write("\n\n The rouge scores on summary file are \n {0}".format(rougescores,2)) #store the results 

  basicfile="original.txt"
  #guj_clean_text(filename,basicfile) # Get the original file in cleaned form for computing compression ratio. 
  guj_remove_newlines(filename,basicfile) 
  
  sent_compression,word_compression,totaloriginalsentences,totaloriginalwords,totalsumsentences,totalsumwords=compression_statistics(summary_filename,basicfile) #to store the compression ratio analysis
  analyzedfile_obj.write("\n **********************Compression Analysis at {0} % extraction is as follows *********************** ".format(percent))
  analyzedfile_obj.write("\n Total sentences in original text were {0} ".format(totaloriginalsentences)) 
  analyzedfile_obj.write("\n Total sentences in summary text are {0} ".format(totalsumsentences))
  analyzedfile_obj.write("\n Sentences compression ratio is {0} \n".format(sent_compression,2))
  analyzedfile_obj.write(" \n\n Total words in original text were {0} ".format(totaloriginalwords))
  analyzedfile_obj.write(" \n total words in summary text are {0} ".format(totalsumwords))
  analyzedfile_obj.write("\n Words compression ratio is {0} ".format(word_compression,2)) 


{'rouge-1': {'f': 0.8066914448147483, 'p': 0.7977941176470589, 'r': 0.8157894736842105}, 'rouge-2': {'f': 0.7350746218662982, 'p': 0.7269372693726938, 'r': 0.7433962264150943}, 'rouge-l': {'f': 0.8289473634288436, 'p': 0.7974683544303798, 'r': 0.863013698630137}}


In [153]:
analyzedfile_obj.close()

rougescorefilehandle.close()